In [28]:
import pandas as pd
import requests
import random

In [29]:
data = pd.read_csv('/Users/37498/Desktop/MA_Milestone2_group5/ma_group_project/data/isp_data.csv').iloc[:,1:]

,account_id,payment_amount,payment_method,payment_date
0,1217,9000.0,Cash Payment,2017-07-25
1,365,12000.0,Cash Payment,2017-07-25
2,925,6000.0,Cash Payment,2017-07-25
3,529,6000.0,Cash Payment,2017-07-25
4,156,6000.0,Cash Payment,2017-07-25


In [ ]:
# base_url = "http://0.0.0.0:8000"
base_url = "http://127.0.0.1:8000"
# base_url = "http://192.168.0.1:8000"

unique_customers = data['account_id'].unique()
account_to_customer_id = {}

for account_id in unique_customers:
    # Generating the age and gender for our customers
    age = random.randint(20, 70)
    gender = random.choice(["Male", "Female"])

    # Insert our customer data into Dim_Customer
    customer_data = {
        "age": age,
        "gender": gender
    }
    response = requests.post(f"{base_url}/customer/", json=customer_data)
    if response.status_code != 200:
        print(f"Failed to insert customer {account_id}. Response: {response.text}")
        continue

    customer_id = response.json()['customer_id']
    account_to_customer_id[account_id] = customer_id

# Finally add customer_id to the dataset
data['customer_id'] = data['account_id'].map(account_to_customer_id)

In [ ]:
unique_payment_methods = data['payment_method'].unique()
payment_to_payment_id = {}

for payment in unique_payment_methods:
    # Insert payment method into Dim_Payment_Method
    payment_data = {
        "payment_method_name": payment
    }
    response = requests.post(f"{base_url}/payment_method/", json=payment_data)
    if response.status_code != 200:
        print(f"Failed to insert payment method {payment}. Response: {response.text}")
        continue

    payment_method_id = response.json()['payment_method_id']
    payment_to_payment_id[payment] = payment_method_id
    
# Add payment_method_id to the dataset
data['payment_method_id'] = data['payment_method'].map(payment_to_payment_id)

In [ ]:
# Insert each row into the Fact_Transacation table
for index, row in data.iterrows():
    transaction_data = {
        "customer_id": row['customer_id'],
        "payment_method_id": row['payment_method_id'],
        "transaction_date": row['payment_date'],
        "amount": row['payment_amount']
    }
    response = requests.post(f"{base_url}/transaction/", json=transaction_data)
    if response.status_code != 200:
        print(f"Failed to insert transaction for row {index}. Response: {response.text}")